In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

plt.style.use('seaborn-white')
plt.rc('figure', dpi=100, figsize=(7, 5))
plt.rc('font', size=12)

import warnings
warnings.simplefilter('ignore')

# Lecture 25 – Decision Trees, Grid Search, and Multicollinearity

## DSC 80, Spring 2022

### Announcements

- Project 4 is due **tomorrow at 11:59PM**.
    - IMPORTANT: See [this post](https://campuswire.com/c/G325FA25B/feed/1511) for more public tests for Question 5.
- Lab 9 is due on **Tuesday, May 31st at 11:59PM** (note the later deadline).
- Discussion 8 (the final discussion of the quarter) is tonight.
- Project 3 grades are released, and the Grade Report has been updated!
- Project 5 will be released on Friday.
- The Final Exam is on **Saturday, June 4th from 11:30AM-2:30PM in-person**!
    - You may bring 2 two-sided cheat sheets.
    - More details soon.

### Agenda
- Decision trees 🌲 and grid searching.
- Multicollinearity.

### Recap: Generalization

1. Split the data into two sets: <span style='color: blue'><b>training</b></span> and <span style='color: orange'><b>test</b></span>.

2. Use only the <span style='color: blue'><b>training</b></span> data when designing, training, and tuning the model.
    - Use <span style='color: green'><b>cross-validation</b></span> to choose hyperparameters and estimate the model's ability to generalize.
    - Do not ❌ look at the <span style='color: orange'><b>test</b></span> data in this step!
    
3. Commit to your final model and train it using the entire <span style='color: blue'><b>training</b></span> set.

4. Test the data using the <span style='color: orange'><b>test</b></span> data. If the performance (e.g. RMSE) is not acceptable, return to step 2.

5. Finally, train on **all available data** and ship the model to production! 🛳

🚨 This is the process you should **always** use! 🚨 

### Discussion Question 🤔

We won't answer this question in class, but **it's a good exam-prep question!**

- Suppose you have a training dataset with 1000 rows.
- You want to decide between 20 hyperparameters for a particular model.
- To do so, you perform 10-fold cross-validation.
- **How many times is the first row in the training dataset (`X.iloc[0]`) used for training a model?**

## Example: Decision trees 🌲 and grid searching

<center><img src='imgs/taxonomy.png' width=50%></center>

Decision trees can be used for both regression and classification. We will start by discussing their use in **classification**.

### Example: Predicting diabetes

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')
diabetes.head()

In [ ]:
diabetes['Outcome'].value_counts()

For illustration, we'll use `'Glucose'` and `'BMI'` to predict whether or not a patient has diabetes (the response variable is in the `'Outcome'` column).

### Building a decision tree

Let's build a decision tree and interpret the results. But first, a train-test split:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes[['Glucose', 'BMI']], 
                                                    diabetes['Outcome'],
                                                    random_state=1)

The relevant class is `DecisionTreeClassifier`, from `sklearn.tree`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Note that we `fit` it the same way we `fit` earlier estimators.

_You may wonder what `max_depth=2` does – more on this soon!_

In [ ]:
dt = DecisionTreeClassifier(max_depth=2)

In [ ]:
dt.fit(X_train, y_train)

### Visualizing decision trees

Our fit decision tree is like a "flowchart", made up of a series of questions.

<span style='color: orange'><b>Class 0 (orange) is "no diabetes"</b></span>; <span style='color: blue'><b>Class 1 (blue) is "diabetes"</b></span>.

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(10, 5))
plot_tree(dt, feature_names=X_train.columns, class_names=['no', 'yes'], 
               filled=True, rounded=True, fontsize=15, impurity=False);

- To **classify a new data point**, we start at the top and answer the first question (i.e. "Glucose <= 129.5").
- If the answer is "Yes", we move to the left branch, otherwise we move to the right branch.
- We repeat this process until we end up at a leaf node, at which point we predict the most common class in that node.
    - Note that each node has a `value` attribute, which describes the number of **training** individuals of each class that fell in that node.

In [ ]:
# Note that the left node at depth 2 has a `value` of [304, 78]
y_train.loc[X_train[X_train['Glucose'] <= 129.5].index].value_counts()

### Evaluating classifiers

The most common evaluation metric in classification is **accuracy**:

$$\text{accuracy} = \frac{\text{# data points classified correctly}}{\text{# data points}}$$

In [ ]:
(dt.predict(X_train) == y_train).mean()

The `score` method of a classifier computes accuracy by default.

In [ ]:
# Training accuracy – same number as above
dt.score(X_train, y_train)

In [ ]:
# Testing accuracy
dt.score(X_test, y_test)

### Some questions...

- How did `sklearn` decide what questions to ask?

- Can we ask more questions (i.e. build a **deeper** tree)?

### Training a decision tree

When we ask a question, we are effectively **splitting** a node into two children – the "yes" child and the "no" child.

Suppose the distribution within a node looks like this (colors represent classes):

<center>🔵🔵🔵🔵🔵🔵🔵🔴🔴🔴</center>

Question A **splits** the node like this:
- "Yes": 🔵🔵🔵🔴🔴
- "No": 🔵🔵🔵🔵🔴

Question B **splits** the node like this:
- "Yes": 🔵🔵🔵🔵🔵🔵
- "No": 🔵🔴🔴🔴

**Which question is "better"?**

Question B, because there is "less uncertainty" in the resulting nodes after splitting by Question B than there is after splitting by Question A. There are two common techniques for quantifying "uncertainty":
- Gini impurity (the default in `sklearn`).
- Entropy (from information theory).

Not the focus of our course, but read more!

### Tree depth

Decision trees are trained by **recursively** picking the best split until:
- all "leaf nodes" only contain training examples from a single class (good), or
- it is impossible to split leaf nodes any further (not good).

By default, there is no "maximum depth" for a decision tree. As such, without restriction, decision trees tend to be very deep.

In [ ]:
dt_no_max = DecisionTreeClassifier()
dt_no_max.fit(X_train, y_train)

A decision tree fit on our training data has a depth of around 20! (It is so deep that `tree.plot_tree` errors when trying to plot it.)

In [ ]:
dt_no_max.tree_.max_depth

At first, this tree seems "better" than our tree of depth 2, since its training accuracy is much much higher:

In [ ]:
dt_no_max.score(X_train, y_train)

In [ ]:
# Depth 2 tree
dt.score(X_train, y_train)

But recall, we truly care about **test set performance**, and this decision tree has **worse accuracy on the test set than our depth 2 tree**.

In [ ]:
dt_no_max.score(X_test, y_test)

In [ ]:
# Depth 2 tree
dt.score(X_test, y_test)

### Decision trees and overfitting

- Decision trees have a tendency to overfit. **Why is that?**

- Unlike linear classification techniques (like logistic regression or SVMs), **decision trees are non-linear**.
    - They are also "non-parametric" – there are no $w^*$s to learn.

- While being trained, decision trees ask enough questions to effectively **memorize** the correct response values in the training set. However, the relationships they learn are often overfit to the noise in the training set, and don't generalize well.

- **Fact:** A decision tree whose depth is not restricted will achieve 100% accuracy on any training set, as long as there are no "overlapping values" in the training set.
    - Two values overlap when they have the same features $x$ but different response values $y$ (e.g. if two patients have the same glucose levels and BMI, but one has diabetes and one doesn't).

- **One solution:** Make the decision tree "less complex" by limiting the maximum depth.

Since `sklearn.tree`'s `plot_tree` can't visualize extremely large decision trees, let's create and visualize some smaller decision trees.

In [ ]:
trees = {}
for d in [2, 4, 8]:
    trees[d] = DecisionTreeClassifier(max_depth=d, random_state=1)
    trees[d].fit(X_train, y_train)
    
    plt.figure(figsize=(15, 5), dpi=100)
    plot_tree(trees[d], feature_names=X_train.columns, class_names=['no', 'yes'], 
               filled=True, rounded=True, impurity=False)
    
    plt.show()

As tree depth increases, complexity increases.

**Question:** What is the right maximum depth to choose?

### Hyperparameters for decision trees

- `max_depth` is a hyperparameter for `DecisionTreeClassifier`.
- There are many more hyperparameters we can tweak; look at [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for examples.
    - `min_samples_split`: The minimum number of samples required to split an internal node.
    - `criterion`: The function to measure the quality of a split (`'gini'` or `'entropy'`).
- **We need an efficient technique for trying different combinations of hyperparameters!**

### Grid search

`GridSearchCV` takes in:
- an **un-`fit`** instance of an estimator, and
- a **dictionary** of hyperparameter values to try,

and performs $k$-fold cross-validation to find the **combination of hyperparameters with the best average validation performance**.

In [ ]:
from sklearn.model_selection import GridSearchCV

The following dictionary contains the values we're considering for each hyperparameter. (We're using `GridSearchCV` with 3 hyperparameters, but we could it with even just a single hyperparameter.)

In [ ]:
hyperparameters = {
    'max_depth': [2, 3, 4, 5, 7, 10, 13, 15, 18, None], 
    'min_samples_split': [2, 3, 5, 7, 10, 15, 20],
    'criterion': ['gini', 'entropy']
}

Note that there are 140 **combinations** of hyperparameters we need to try. We need to find the **best combination** of hyperparameters, not the best value for each hyperparameter individually.

In [ ]:
len(hyperparameters['max_depth']) * len(hyperparameters['min_samples_split']) * len(hyperparameters['criterion'])

`GridSearchCV` needs to be instantiated and `fit`.

In [ ]:
searcher = GridSearchCV(DecisionTreeClassifier(), hyperparameters, cv=5)

In [ ]:
searcher.fit(X_train, y_train)

After being `fit`, the `best_params_` attribute provides us with the best combination of hyperparameters to use.

In [ ]:
searcher.best_params_

All of the intermediate results – validation accuracies for each fold, mean validation accuaries, etc. – are stored in the `cv_results_` attribute:

In [ ]:
searcher.cv_results_['mean_test_score'] # array of length 140

In [ ]:
# Rows correspond to folds, columns correspond to hyperparameter combinations
pd.DataFrame(np.vstack([searcher.cv_results_[f'split{i}_test_score'] for i in range(5)]))

Note that the above DataFrame tells us that 5 * 140 = 700 models were trained in total!

**Question:** How is the following line of code making predictions?

In [ ]:
searcher.predict(X_train)[:5]

Which model's testing accuracy is shown below?

In [ ]:
searcher.score(X_test, y_test)

### Key takeaways

- Decision trees are trained by finding the best questions to ask using the features in the training data. A good question is one that isolates classes as much as possible.
- Decision trees have a tendency to overfit to training data. One way to mitigate this is by restricting the maximum depth of the tree.
- To efficiently find hyperparameters through cross-validation, use `GridSearchCV`.
    - Specify which values to try for each hyperparameter, and `GridSearchCV` will try all **unique combinations of hyperparameters** and return the combination with the best average validation performance.
    - `GridSearchCV` is not the only solution – see [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) if you're curious.

## Multicollinearity

### Motivating example

Suppose we fit a simple linear regression model that uses **height in inches** to predict **weight in pounds**.

$$\text{predicted weight} = w_0 + w_1 \cdot \text{height (inches)}$$

In [ ]:
people = pd.read_csv('data/SOCR-HeightWeight.csv').drop('Index', axis=1)
people.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(people[['Height(Inches)']], 
                                                            people['Weight(Pounds)'], 
                                                            random_state=1)

In [ ]:
lr_one_feat = LinearRegression()
lr_one_feat.fit(X_train_1, y_train_1)

$w_0^*$ and $w_1^*$ are shown below, along with the model's **testing** RMSE.

In [ ]:
lr_one_feat.intercept_, lr_one_feat.coef_

In [ ]:
rmse_one_feat = mean_squared_error(y_test_1, 
                                   lr_one_feat.predict(X_test_1), 
                                   squared=False)
rmse_one_feat

Now, suppose we fit another regression model, that uses **height in inches** AND **height in centimeters** to predict weight.

$$\text{predicted weight} = w_0 + w_1 \cdot \text{height (inches)} + w_2 \cdot \text{height (cm)}$$

In [ ]:
people['Height(cm)'] = people['Height(Inches)'] * 2.54 # 1 inch = 2.54 cm

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(people[['Height(Inches)', 'Height(cm)']], 
                                                            people['Weight(Pounds)'], 
                                                            random_state=1)

In [ ]:
lr_two_feat = LinearRegression()
lr_two_feat.fit(X_train_2, y_train_2)

What are $w_0^*$, $w_1^*$, $w_2^*$, and the model's testing RMSE?

In [ ]:
lr_two_feat.intercept_, lr_two_feat.coef_

In [ ]:
rmse_two_feat = mean_squared_error(y_test_2, 
                                   lr_two_feat.predict(X_test_2), 
                                   squared=False)
rmse_two_feat

**Observation:** The intercept is the same as before (roughly -81.17), as is the testing RMSE. However, the coefficients on `'Height(Inches)'` and `'Height(cm)'` are massive in size!

What's going on?

### Redundant features

Let's use simpler numbers for illustration. Suppose in the first model, $w_0^* = -80$ and $w_1^* = 3$.

$$\text{predicted weight} = -80 + 3 \cdot \text{height (inches)}$$

In the second model, we have:

$$\begin{align*}\text{predicted weight} &= w_0^* + w_1^* \cdot \text{height (inches)} + w_2^* \cdot \text{height (cm)} \\ &= w_0^* + w_1^* \cdot \text{height (inches)} + w_2^* \cdot \big( 2.54^* \cdot \text{height (inches)} \big) \\ &= w_0^* + \left(w_1^* + 2.54 \cdot w_2^* \right) \cdot \text{height (inches)} \end{align*}$$

In the first model, we already found the "best" intercept and slope in a linear model that uses height in inches to predict weight ($-80$ and $3$).

So, as long as $w_1^* + 2.54 \cdot w_2^* = 3$ in the second model, the second model's training predictions will be the same as the first, and hence they will also minimize RMSE.

### Infinitely many parameter choices

**Issue:** There are an infinite number of $w_1^*$ and $w_2^*$ that satisfy $w_1^* + 2.54 \cdot w_2^* = 3$!

$$\text{predicted weight} = -80 - 10 \cdot \text{height (inches)} + \frac{13}{2.54} \cdot \text{height (cm)}$$

$$\text{predicted weight} = -80 + 10 \cdot \text{height (inches)} - \frac{7}{2.54} \cdot \text{height (cm)}$$

- Both prediction rules look very different, but actually make the same predictions on the training set.
- `lr.coef_` could return either set of coefficients, or any other of the infinitely many options. 
- But neither set of coefficients is **has any meaning!**

In [ ]:
(-80 - 10 * people.iloc[:, 0] + (13 / 2.54) * people.iloc[:, 2]).head()

In [ ]:
(-80 + 10 * people.iloc[:, 0] - (7 / 2.54) * people.iloc[:, 2]).head()

### Multicollinearity

- Multicollinearity occurs when features in a regression model are **highly correlated** with one another.
    - In other words, multicollinearity occurs when **a feature can be predicted using a linear combination of other features, fairly accurately**.

- When multicollinearity is present in the features, the **coefficients in the model** are uninterpretable – they have no meaning.
    - A "slope" represents "the rate of change of $y$ with respect to a feature", when all other features are held constant – but if there's multicollinearity, you can't hold other features constant.

- **Note: Multicollinearity doesn't impact a model's predictions!**
    - It doesn't impact a model's ability to generalize to unseen data.
    - If features are multicollinear in the training data, they will probably be multicollinear in the test data too.

- **Solutions:**
    - Manually remove highly correlated features.
    - Use a dimensionality reduction technique (such as PCA) to automatically reduce dimensions.

### One-hot encoding and multicollinearity

When we one-hot encode categorical features, we create several **redundant** columns.

In [ ]:
tips = sns.load_dataset('tips')
tips_features = tips.drop('tip', axis=1)
tips_features.head()

Aside: You can use `pd.get_dummies` in EDA, but **don't** use it for modeling (instead, use `OneHotEncoder`, which works with `Pipeline`s).

In [ ]:
X = pd.get_dummies(tips_features)
X.head()

Remember that under the hood, `LinearRegression()` creates a **design matrix** that has a column of all ones (for the intercept term). Let's add that column above for demonstration.

In [ ]:
X['all_ones'] = 1
X.head()

Now, many of the above columns **can be written as linear combinations of other columns**!
- We don't need `'sex_Male'` – its value is just `'all_ones'` - `'sex_Female'`.
- We don't need `'smoker_Yes'` – its value is just `'all_ones'` - `'smoker_No'`.
- We don't need `'time_Lunch'` – its value is just `'all_ones'` - `'time_Dinner'`.
- We don't need `'day_Thur'` – its value is just `'all_ones'` - (`'day_Fri'` + `'day_Sat'` + `'day_Sun'`).

Note that if we get rid of the four redundant columns above, the **rank** of our design matrix – that is, the number of linearly independent columns it has – does not change (and so the "predictive power" of our features don't change either).

In [ ]:
np.linalg.matrix_rank(X)

In [ ]:
np.linalg.matrix_rank(X.drop(columns=['sex_Male', 'smoker_Yes', 'time_Lunch', 'day_Thur']))

However, without the redundant columns, there is only a single unique set of optimal parameters $w^*$, and the multicollinearity is no more.

**Aside:** Most one-hot encoding techniques (including `OneHotEncoder`) have an in-built `drop` argument, which allow you to specify that you'd like to drop **one column per categorical feature**.

In [ ]:
pd.get_dummies(tips_features, drop_first=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder(drop='first')
ohe.fit_transform(tips_features[['sex', 'smoker', 'day', 'time']]).toarray()

The above array only has $(2-1) + (2-1) + (4-1) + (2-1) = 6$ columns, rather than $2 + 2 + 4 + 2 = 10$, since we dropped 1 per categorical column in `tips_features`.

### Key takeaways

- Multicollinearity is present in a linear model when one feature can be accurately predicted using one or more other features.
    - In other words, it is present when a feature is **redundant**.
- Multicollinearity doesn't pose an issue for prediction; it doesn't hinder a model's ability to generalize. Instead, it renders the **coefficients** of a linear model meaningless.
- Multicollinearity is present when performing one-hot encoding; a solution is to drop **one one-hot-encoded column for each original categorical feature**.

## Summary, next time

### Summary

- See the individual sections for more specific "key takeaways".
- **Next time:** 
    - Using text features in a predictive model.
    - Metrics for measuring the performance of classifiers other than accuracy.